In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

import shap
import warnings
import logging

warnings.filterwarnings("ignore")
logger = logging.getLogger('shap')
logger.disabled = True

# Build Autoencoder model

In [ ]:
def train_model(X_train, nb_epoch=1000, batch_size=64):

    input_dim = X_train.shape[1]

    input_layer = Input(shape=(input_dim, ))

    encoder = Dense(int(input_dim/2), activation="relu", activity_regularizer=regularizers.l1(10e-7))(input_layer)

    encoder = Dense(int(input_dim/4), activation="relu", kernel_regularizer=regularizers.l2(10e-7))(encoder)

    decoder = Dense(int(input_dim/2), activation='relu', kernel_regularizer=regularizers.l2(10e-7))(encoder)
    
    decoder = Dense(input_dim, activation='sigmoid', kernel_regularizer=regularizers.l2(10e-7))(decoder)

    autoencoder = Model(inputs=input_layer, outputs=decoder)

    autoencoder.summary()

    # Configures the learning process of the network
    autoencoder.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])

    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    # Train the autoencoder based on the best epoch, returns history object
    autoencoder.fit(X_train, X_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True, 
                    validation_split=0.1, verbose=2, callbacks=[earlystopper])
    
    return autoencoder

# Find anomalies and return them sorted by mse

In [ ]:
def get_top_anomaly_to_explain(autoencoder, X, n_anomalies_to_explain):
    predictions = autoencoder.predict(X)
    square_errors = np.power(X - predictions, 2)
    mse_series = pd.Series(np.mean(square_errors, axis=1))
    
    most_anomal_trx = mse_series.sort_values(ascending=False)
    columns=["id", "mse_all_columns"]
    columns.extend(["squared_error_" + x  for x in list(X.columns)])
    items = []
    for x in most_anomal_trx.iteritems():
        item = [x[0], x[1]]
        item.extend(square_errors.loc[x[0]])
        items.append(item)

    df_anomalies = pd.DataFrame(items, columns=columns)
    df_anomalies.set_index('id', inplace=True)
    
    top_anomalies_to_explain = df_anomalies.head(n_anomalies_to_explain).index
    return top_anomalies_to_explain

# Explainability

In [ ]:
def relevant_features(total_squared_error, df_with_errors): # return features that cause 50% of the sum of mse
    error = 0
    for num_of_features, index in enumerate(df_with_errors.index):
        error += df_with_errors.loc[index, 'err']
        if error >= 0.5 * total_squared_error:
            break
    return num_of_features + 1

def get_all_user_trx(X, num_of_trx):
    return X.head(num_of_trx)

def get_err_per_trx(current_trx):
    prediction = autoencoder.predict(np.array([[current_trx]])[0])[0]
    square_errors = np.power(current_trx - prediction, 2)
    trx_err_in_row = pd.DataFrame({'col_name': square_errors.index, 'err': square_errors}).reset_index(drop=True)
    total_mse = np.mean(square_errors)
    trx_err_in_row.sort_values(by='err', ascending=False, inplace=True)
    return trx_err_in_row, total_mse

def get_highest_values(data_set): # return features that higher than mean shap\lime value
    n_largest_data_frame = pd.DataFrame()
    n_explaining_features = {}
    for i in range(len(data_set)):
        values = data_set.iloc[i]
        mean_val = np.mean(values)
        n_features = 0
        for j in range(len(values)):
            if values[j] > mean_val:
                n_features += 1
        n_explaining_features[i] = n_features
        curr_n_largest = data_set[i:i + 1].stack().nlargest(n_features)
        n_largest_data_frame = pd.concat([n_largest_data_frame, curr_n_largest], axis=0)
    return n_largest_data_frame, n_explaining_features

def f_explain_features(X):
    pred = autoencoder.predict(X)[:,counter]
    return pred 
    
def get_features_set(set_explaning_features):
    features_set = []
    for feature_name in np.concatenate(set_explaning_features):
        if feature_name not in features_set:
            features_set.append(feature_name)
    return features_set

def get_explaining_features(top_trx_to_explain, autoencoder, X_train, X_explain):
    global counter
    trx_counter = 0

    all_set_explaning_features = {}

    for trx_number in top_trx_to_explain:

        trx_counter = trx_counter + 1
        print('trx_counter', trx_counter)
        print(trx_number)

        current_trx = X_explain.loc[trx_number]
        prediction_trx = autoencoder.predict(np.array([[current_trx]])[0])

        df_err, total_mse = get_err_per_trx(current_trx)
        num_of_features = relevant_features(total_mse*df_err.shape[0], df_err)

        df_top_err = df_err.head(num_of_features)
        all_set_explaning_features[trx_number] = []
        values_all_features = [[] for num in range(num_of_features)]

        backgroungd_set = get_all_user_trx(X_train, 200).values
        for i in range(num_of_features):
            counter = df_top_err.index[i]
            explainer = shap.KernelExplainer(f_explain_features, backgroungd_set)
            shap_values = explainer.shap_values(current_trx, nsamples='auto')
            values_all_features[i] = shap_values 

        values_all_features = np.fabs(values_all_features)

        contributing_data_frame = pd.DataFrame(data=values_all_features, columns=X_train.columns)  
        highest_contributing, n_explaining_features = get_highest_values(contributing_data_frame)

        for idx_explained_feature in range(num_of_features):
            set_explaning_features = [x[1] for x in highest_contributing.index if x[0] == idx_explained_feature]
            explaines_feature_index = df_top_err.index[idx_explained_feature]
            set_explaning_features.insert(0, X_train.columns[explaines_feature_index])
            
            all_set_explaning_features[trx_number].append(set_explaning_features)

        all_set_explaning_features[trx_number] = get_features_set(all_set_explaning_features[trx_number])
       
        
    return all_set_explaning_features


def explain_data(X_train, X_explain, autoencoder=None, n_anomalies_to_explain=100):
    if autoencoder == None:
        autoencoder = train_model(X_train)

    top_trx_to_explain = get_top_anomaly_to_explain(autoencoder, X_explain, n_anomalies_to_explain)
    
    counter = 0
    all_set_explaning_features_shap = get_explaining_features(top_trx_to_explain, autoencoder, X_train, X_explain)
    return all_set_explaning_features_shap

# Example of run

# Generate random data

In [ ]:
def create_random_binary_data(n_records=1000000, n_anomalies=100, n_features=20):
    features_names = []
    for feature in range(1, n_features+1):
        features_names.append('feature_' + str(feature))

    data = np.random.randint(2, size=(n_records, n_features)) 
    data_df = pd.DataFrame(data=data, columns=features_names)
    data_df['feature_1_2'] = data_df['feature_1'] & data_df['feature_2']
    data_df['feature_3_4'] = data_df['feature_3'] | data_df['feature_4']
    data_df['class'] = 0

    idx = data_df.sample(n=n_anomalies).index

    for i in idx:
        rand_num = np.random.rand(1)[0]
        if rand_num < 0.5:
            val = data_df.loc[i, 'feature_1_2']
            data_df.loc[i, 'feature_1_2'] = 1 - val
            data_df.loc[i, 'class'] = '1_2'
        else:
            val = data_df.loc[i, 'feature_3_4']
            data_df.loc[i, 'feature_3_4'] = 1 - val
            data_df.loc[i, 'class'] = '3_4'

    X = data_df.iloc[:,:-1]
    y = data_df.iloc[:, -1]

    train_idx = y[y==0].index.values
    test_idx = y[y!=0].index.values

    X_train = X.iloc[train_idx]
    y_train = y[train_idx]
    X_test = X.iloc[test_idx]
    y_test = y[test_idx]

    return X_train, y_train, X_test, y_test

# Get explnation of anomalies on generated random data

In [ ]:
X_train, y_train, X_explain, y_explain = create_random_binary_data()
explain_data(X_train, X_explain)